# Support Vector Machine

> “Nothing is more practical than a good theory.”
> -Kurt Lewin, Vladimir Vapnik



클로드 섀넌 등 위대한 석학들이 거쳐간 벨 연구소에서 (그 당시의) 머신러닝 분류기법의 4번타자인 Support Vector Machine 이 1995년 블라디미르 바프닉의 연구 끝에 탄생했다.

Computational Complexity 에 대한 한계가 명확한 신경망을 제치고 SVM 은 효과적인 모델로 자리잡았다. 고차원에서 좋은 성능을 보이고, 샘플 수 보다 차원 수가 높아도 준수한 성늘을 보이고, 본질적으로 Computational Complexity 가 낮은 SVM 에 대한 연구는 계속되었다.

하지만 SVM 을 완성시킨 바프닉이 말한 것 처럼 2000년대 이후 SVM은 신경망을 뛰어넘지 못했고, 딥러닝이 대세가 된 현재 SVM 의 practical application 은 찾기 힘들다.

그렇다고 SVM 을 고대의 유물취급하기엔 kernel trick 등 효과적인 이론들을 머신러닝에 도입시켰고, ‘근본’ 모델 중 하나이기에 배울 수 있는게 많다. 바프닉의 말처럼 좋은 이론만큼 실용적인 것은 없기 때문이다.



In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
import cvxopt
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix